##### following ipyrad cookbook "parallelized STRUCTURE analyses on unlinked SNPs", in addition to some minor changes in formatting found at https://radcamp.github.io/NYC2018/05_STRUCTURE_API.html
##### on my part, this is intended to be a first-pass, in terms of analyses . more detailed analyses, additional parameter estimates, and/or more information might be more useful for determining the "appropriate number of K"

In [ ]:
import ipyrad.analysis as ipa
import ipyparallel as ipp
import toyplot

##### to start an ipcluster instance in a separate terminal, go to the Jupyter dashboard, click on 'new', click on 'terminal', and enter "ipcluster start" to begin a local cluster on the computer

In [ ]:
# set up a parallel client
ipyclient = ipp.Client()
print "Connected to {} cores".format(len(ipyclient))

## Set up STRUCTURE input files

In [ ]:
# location of structure formatted file
strfile = "/Users/sjjacobs/Desktop/data_to_mess_with/subset1_outfiles/subset1.str"
# path to mapfile, to sample unlinked SNPs
mapfile = "/Users/sjjacobs/Desktop/data_to_mess_with/subset1_outfiles/subset1.snps.map"
# outfile directory
workdir = "/Users/sjjacobs/Desktop/data_to_mess_with/analysis_strucutre_morereps/"

In [ ]:
# make a structure object
struct = ipa.structure(name="structure-test",
                      data = strfile,
                      mapfile = mapfile, 
                      workdir = workdir)

In [ ]:
# set mainparams for object
struct.mainparams.burnin = 10000
struct.mainparams.numreps = 100000

# see all mainparams
print struct.mainparams

# see or set extraparams
print struct.extraparams

In [ ]:
# range of K-values to test
tests = [2,3,4,5,6]

## Submit the batch of jobs to the ipcluster instance initiated earlier

In [ ]:
# submit a batch of 20 replicates for each value of K
for kpop in tests:
    struct.run(
        kpop=kpop,
        nreps=20,
        seed=12345,
        ipyclient=ipyclient)

In [ ]:
# check in on the status of the runs ... patience, patience!
struct.asyncs[:]

## Use CLUMPP to permute the output of our independent runs for each value of K

In [ ]:
# set some clump parameters
struct.clumppparams.repeats = 10000
struct.clumppparams

In [ ]:
# rum clumpp for each value of K
tables = struct.get_clumpp_table(tests)

## Calculate statistics to determine best value of K

In [146]:
# return the evanno table with deltaK
struct.get_evanno_table(tests)


,Nreps,deltaK,estLnProbMean,estLnProbStdev,lnPK,lnPPK
2,24,0.000,-14125.142,189.154,0.000,0.000
3,20,353.711,-9347.605,186.450,4777.537,65949.357
4,20,0.797,-70519.425,44429.442,-61171.820,35408.210
5,20,0.623,-96283.035,67534.912,-25763.610,42046.345
6,20,0.000,-80000.300,60922.628,16282.735,0.000


In [147]:
# make a custom order for the output - useful if you're comparing to a tree and want the names ordered in the same way as your phylogeny
myorder = ["FZ266_paniculata",
          "FZ271_paniculata",
          "FZ200_paniculata",
          "JW11465_paniculata",
          "FZ236_piurensis",
          "FZ238_piurensis",
          "WP152_farinacea",
          "WP160_farinacea",
          "LF61_bifida",
          "LF72_megapotamica",
          "LF59_ledifolia",
          "LF57_ledifolia"
          ]
print "custom ordering"
# use the index after 'tables' to examine the ancestry components for K=x
# .loc[] notation specifies to fetch from the table by row
print tables[3].loc[myorder]

custom ordering
                      0      1      2
FZ266_paniculata    0.0  1.000  0.000
FZ271_paniculata    0.0  1.000  0.000
FZ200_paniculata    0.0  1.000  0.000
JW11465_paniculata  0.0  0.994  0.006
FZ236_piurensis     0.0  0.000  1.000
FZ238_piurensis     0.0  0.000  1.000
WP152_farinacea     1.0  0.000  0.000
WP160_farinacea     1.0  0.000  0.000
LF61_bifida         1.0  0.000  0.000
LF72_megapotamica   1.0  0.000  0.000
LF59_ledifolia      1.0  0.000  0.000
LF57_ledifolia      1.0  0.000  0.000


## Visualize the different popualtion STRUCTURE schemes
### Here, bars are in the same order as the table above

In [149]:
for kpop in tests:
    # parse outfile to a table and re-order it
    table = tables[kpop]
    table = table.loc[myorder]
    
    # plot barplot with hover
    canvas, axes, mark = toyplot.bars(
                            table,
                            width=400,
                            height=200,
                            style={"stroke": toyplot.color.near_black},
                            )


0 4 8 12 0.0 0.5 1.0

0 4 8 12 0.0 0.5 1.0

0 4 8 12 0.0 0.5 1.0

0 4 8 12 0.0 0.5 1.0

0 4 8 12 0.0 0.5 1.0

## A slightly fancier plot, saved to file

In [152]:
def fancy_plot(table):
    import toyplot
    # some extra styling with css
    style = {"stroke":toyplot.color.near_black,
        "stroke-width":2}
    
    # build barplot
    canvas = toyplot.Canvas(width=600, height=250)
    axes = canvas.cartesian(bounds=("5%","95%","5%","45%"))
    axes.bars(table, style=style)
    
    # add names to x-axis
    ticklabels = [i for i in table.index.tolist()]
    axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
    axes.x.ticks.labels.angle = -60
    axes.x.ticks.show = True
    axes.x.ticks.labels.offset = 10
    axes.x.ticks.labels.style = {"font-size": "12px"}
    axes.x.spine.style = style
    axes.y.show = False
    
    import toyplot.svg
    import toyplot.pdf
    toyplot.svg.render(canvas, "struct.svg")
    toyplot.pdf.render(canvas, "struct.pdf")

    return canvas

In [153]:
# save plots for your favorite value of K
table = struct.get_clumpp_table(kvalues=3)
table = table.loc[myorder]
fancy_plot(table)


[K3] 20/20 results permuted across replicates (max_var=0).


Exception: A ghostscript executable is required.